# Solving Simply Supported Beam w/ Mesh & Beam Classes

This notebook ideally serves as a comprehensive guide going over the Mesh and Beam classes developed for Assignment 2 Part 1. These classes are used to construct and solve linear beam systems. This notebook will specifically go over a simple example that will show how to use this to solve any linear beam system.

The problem that we will be analyzing is a simply supported beam with a load directly at the center:

![Problem Diagram](figures/simple_simple_diagram.png)

We will simulate this problem by breaking the beam up into two halves connected at a node where the force is being applied. The two end-nodes will have fixed translational and free rotational boundary conditions while the center node will have free translational and fixed rotational boundary conditions. 

## Class Overview

There are two classes used to construct the truss system: ```Mesh``` and ```Beam```.

The ```Mesh``` class represents a mesh of nodes connected by elements. Each node has a set of degrees of freedom connected x with some action or force f. These nodes are then connected to each other via elements. These object describe how the x and f of each node relate to each other based on the following equation:

```math 
\left[\begin{matrix}\vec{f}_1 \\ \vec{f}_2\end{matrix}\right] = K \left[\begin{matrix}\vec{x}_1 \\ \vec{x}_2\end{matrix}\right] 
```

The Mesh class allows you to gather a bunch of these elements connected together into a single system which can be solved all at once. For our system, x will represent the translation and rotation of our nodes while f will represent the forces and moments in our system

In [5]:
from mesh import Mesh

The ```Beam``` class is used to specifically describe a beam element. The user can pass all the necessary parameters into creating a ```Beam``` object and it will have all the necessary built-in methods to be used with the ```Mesh``` class.

In [6]:
from beam import Beam

Finally, we need to install ```numpy``` since all of our vectors and matrices are stored and passed with ```ndarray```.

In [7]:
import numpy as np

The first thing we do is initialize a ```Mesh```. To do this, all we need to do is specify the number of DoFs at each node. For a beam problem, that number is 6 per node: 3 for each translational DoF and 3 for each rotational DoF.

In [8]:
mesh = Mesh(6)

Next we will define the geometry and loading of our system. This is done through adding nodes to the ```Mesh``` object we created. To start, we define the total force applied at the center node and the total length of the beam:

In [9]:
F = 1
L = 1

Nodes are added to a mesh using four inputs: An integer index starting at 0, the spatial position of the node, the boundary conditions at the node, and optionally any applied forces. 

The boundary condition formatting is as follows:

- If it is any defined number, it specifies the fixed displacement along that DoF
- If it is assigned as ```np.nan```, then it is free and the condition is that the total reaction at the node is zero.

In [10]:
# We begin applying nodes.

# If no applied forces are given, it is assumed to be zero. Displacement format follows [x, y, z, rotx, roty, rotz]
mesh.add_node(0, np.array([0, 0, 0]), np.array([0, 0, 0, np.nan, np.nan, np.nan]))
# Center node. Applied force format follow [Fx, Fy, Fz, Mx, My, Mz]
mesh.add_node(1, np.array([L/2, 0, 0]), np.array([np.nan, np.nan, np.nan, 0, 0, 0]), bf=np.array([0, -F, 0, 0, 0, 0]))
# End node. Same as the first node just at a different position.
mesh.add_node(2, np.array([L, 0, 0]), np.array([0, 0, 0, np.nan, np.nan, np.nan]))

We now need to define our beams. Each beam requires the following parameters:

- Elastic Modulus E
- Constant Cross-Sectional Area A
- Cartestian Second Moments of Area Iy and Iz
- Polar Second Moment of Area J
- Poisson Ratio v

In [11]:
E = 1
A = 1
Iy = 1
Iz = 1
J = 1
v = 0.2

With the constants, we can passed it into the ```Beam``` constructor along with an orientation vector. This is just any vector in the xy-plane and is used to determine the orientation of the beam. One this is done, it can be added to the mesh using the ```add_element``` method.

In [12]:
beam = Beam(E=E, A=A, Iy=Iy, Iz=Iz, J=J, v=v, y=np.array([0, 1, 0]))
# Add the connection to the mesh as a beam. We specify the nodes based on the identifier
mesh.add_element(beam, 0, 1)
mesh.add_element(beam, 1, 2)

Once this is done, we can solve our system to get our displacements and reactions.

In [13]:
x, f = mesh.solve()

print(x)
print(f)

[ 0.          0.          0.          0.          0.         -0.0625
  0.         -0.02083333  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.0625    ]
[0.  0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.  0.  0.  0. ]


This problem has an explicit solution. If we compare them, we can see that the ```Mesh``` and ```Beam``` classes are able to successfully solve the system

In [14]:
tzmax = F*L**2/(16*E*Iz)
ymax = -F*L**3/(48*E*Iz)

d = np.array([
    0, 0, 0,
    0, 0, -tzmax,
    0, ymax, 0,
    0, 0, 0,
    0, 0, 0,
    0, 0, tzmax
])
r = np.array([
    0, F/2, 0,
    0, 0, 0,
    0, 0, 0,
    0, 0, 0,
    0, F/2, 0,
    0, 0, 0
])

print(np.linalg.norm(d - x))
print(np.linalg.norm(r - f))

0.0
0.0
